In [2]:
import numpy as np
from mnist import MNIST
import os
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [4]:
def flattening(X):
    (a,b,c,d) = X.shape
    X = np.reshape(X,(a,b*c*d))
    return X.T

In [5]:
def inv_flattening(X,a,b,c,d):
    X = np.reshape(X,(a,b,c,d))
    return X

In [6]:
def relu(z):
    return np.maximum(0,z)

In [7]:
def relu_back(dA,Z):
    Z[Z>0] = 1
    Z[Z<0] = 0
    return np.multiply(dA,Z)

In [8]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

In [9]:
def softmax(z):
    A = np.exp(z)
    B = np.sum(np.exp(z))
    return A/B

In [10]:
def zero_pad(X, pad):
    X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)), 'constant', constant_values = (0,0))
    return X_pad

In [11]:
def create_mask_from_window(x):

    mask = (x == np.max(x))
    
    return mask

In [12]:
def distribute_value(dz, shape):
    
    (n_H, n_W) = shape
    
    average = dz/(n_H*n_W)
    
    a = np.full((n_H,n_W), average)
    
    return a

In [13]:
def conv_single_step(a_slice_prev, W, b):

    s = np.multiply(a_slice_prev,W)
    
    Z = np.sum(s)
    
    Z = Z+b

    return Z

In [14]:
def costfun(Y,Y_hat):
    (m,n) = Y.shape
    A = np.multiply(Y,np.log(Y_hat))
    cost1 = np.sum(A, axis = 0)
    cost = (-1/m)*np.sum(cost1)
    return cost

In [15]:
def batchnormalization(z):
    mean = np.mean(z)
    std = np.std(z)
    z = np.divide((z-mean),std)
    return z

In [16]:
def batchnorm_forward(x, gamma, beta, eps):

  N, D = x.shape

  #step1: calculate mean
  mu = 1./N * np.sum(x, axis = 0)

  #step2: subtract mean vector of every trainings example
  xmu = x - mu

  #step3: following the lower branch - calculation denominator
  sq = xmu ** 2

  #step4: calculate variance
  var = 1./N * np.sum(sq, axis = 0)

  #step5: add eps for numerical stability, then sqrt
  sqrtvar = np.sqrt(var + eps)

  #step6: invert sqrtwar
  ivar = 1./sqrtvar

  #step7: execute normalization
  xhat = xmu * ivar

  #step8: Nor the two transformation steps
  gammax = gamma * xhat

  #step9
  out = gammax + beta

  #store intermediate
  cache = (xhat,gamma,xmu,ivar,sqrtvar,var,eps)

  return out, cache

In [17]:
def batchnorm_backward(dout, cache):

  #unfold the variables stored in cache
  xhat,gamma,xmu,ivar,sqrtvar,var,eps = cache

  #get the dimensions of the input/output
  N,D = dout.shape

  #step9
  dbeta = np.sum(dout, axis=0)
  dgammax = dout #not necessary, but more understandable

  #step8
  dgamma = np.sum(dgammax*xhat, axis=0)
  dxhat = dgammax * gamma

  #step7
  divar = np.sum(dxhat*xmu, axis=0)
  dxmu1 = dxhat * ivar

  #step6
  dsqrtvar = -1. /(sqrtvar**2) * divar

  #step5
  dvar = 0.5 * 1. /np.sqrt(var+eps) * dsqrtvar

  #step4
  dsq = 1. /N * np.ones((N,D)) * dvar

  #step3
  dxmu2 = 2 * xmu * dsq

  #step2
  dx1 = (dxmu1 + dxmu2)
  dmu = -1 * np.sum(dxmu1+dxmu2, axis=0)

  #step1
  dx2 = 1. /N * np.ones((N,D)) * dmu

  #step0
  dx = dx1 + dx2

  return dx, dgamma, dbeta


In [18]:
def conv_forward(A_prev, W, b, hparameters):
      
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    n_H = int((n_H_prev-f+2*pad)/stride) + 1 
    n_W = int((n_W_prev-f+2*pad)/stride) + 1
    
    Z = np.zeros((m,n_H,n_W,n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):                               
        a_prev_pad = A_prev_pad[i,:,:,:]                               
        for h in range(n_H):                           
            for w in range(n_W):                       
                for c in range(n_C):                   
                    
                    vert_start = h *stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:,:,:,c], b[:,:,:,c])
                                        
       
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [19]:
# Backward convolution 
def conv_backward(dZ, cache):
    
    (A_prev, W, b, hparameters) = cache
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    (m, n_H, n_W, n_C) = dZ.shape
    
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1,1,1,n_C))

    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                       
        
        a_prev_pad = A_prev_pad[i,:,:,:]
        da_prev_pad = dA_prev_pad[i,:,:,:]
        
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):           
                    
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_slice = A_prev_pad[i,vert_start:vert_end,horiz_start:horiz_end,:]

                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        dA_prev[i, :, :, :] = da_prev_pad[pad:-pad,pad:-pad,:]
    
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

In [20]:
def pool_forward(A_prev, hparameters, mode = "max"):
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))              
    
    for i in range(m):                         
        for h in range(n_H):                     
            for w in range(n_W):                 
                for c in range (n_C):            
                    
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_prev_slice = A_prev[vert_start:vert_end,horiz_start:horiz_end,:]
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
        
    cache = (A_prev, hparameters)
    
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

In [21]:
#Backward pooling 
def pool_backward(dA, cache, mode = "max"):
    
    (A_prev, hparameters) = cache
    
    stride = hparameters['stride']
    f = hparameters['f']
    
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    
    dA_prev = np.zeros((A_prev.shape))
    
    for i in range(m):                       
        
        a_prev = A_prev[i,:,:,:]
        
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):           
                    
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    if mode == "max":
                        
                        a_prev_slice = a_prev[vert_start:vert_end,horiz_start:horiz_end,c]
                        
                        mask = create_mask_from_window(a_prev_slice)
        
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += mask * dA[i,h,w,c]
                        
                    elif mode == "average":
                        
                        da = dA[i,h,w,c]
                
                        shape = (f,f)
                        
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += distribute_value(da, shape)
    
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev

In [34]:
np.random.seed(1)
mndata = MNIST('/home/shabaz/Documents/python-mnist/')
X_train, y_train = mndata.load_training() 

X_train = np.array(X_train)  #Data Scaling 
y_train = np.array(y_train)

X_train = X_train.reshape(X_train.shape[0],28,28,1)

X = X_train[:500,:,:,:]
y_train = y_train.reshape(X_train.shape[0],1)
Y = y_train[:500,:]
Y = convert_to_one_hot(Y, 10).T
(n,l,m,o) = X.shape

cost = []

##################### Hyperparameter declaration ############
# for Forward convolution
W = np.random.randn(3,3,1,8)
b = np.random.randn(1,1,1,8)

alpha = 0.001   # for Convolotion part
alpha1 = 0.001  # for first layer of neural network  (RelU)
alpha2 = 0.001  # for second layer of neural network (Softmax)
gamma1,gamma2 = np.ones((1,10)),np.ones((1,10))
beta1,beta2 = np.zeros((1,10)),np.zeros((1,10))
eps = 0.00001 

In [38]:
for i in range(100):
    # for Forward convolution
    hparameters = {"pad" : 1,"stride":1}
    Z, cache_conv = conv_forward(X, W, b, hparameters)

    # Implementing RelU 
    R1 = relu(Z)

    # Pooling 
    hparameters = {"stride" : 2, "f": 2}
    A, cache = pool_forward(R1, hparameters,mode = "max")
    (a,l,m,o) = A.shape

    ######### Hyperparameter for forward and backward ##########
    ############### Weight for neural layers ###################
    W1 = np.random.randn(l*m*o,10)*0.01
    W2 = np.random.randn(10,10)*0.01

    # bias for neural layers
    b1 = np.random.randn(1,1)*0.01
    b2 = np.random.randn(1,1)*0.01

    # Flattening
    S = flattening(A)
    
    ###################### Forward Prop #######################
    ###################### Layer 1 ############################
    Z1 = np.dot(S.T,W1)+b1
    ############# Batch Normalization of first FC #############
    #S = batchnormalization(S)
    Z1, cache1 = batchnorm_forward(Z1, gamma1, beta1, eps)
    A1 = sigmoid(Z1)
    
    ###################### Layer 2 ############################
    Z2 = np.dot(A1,W2)+b2
    ############# Batch Normalization of Second FC ############
    Z2, cache2 = batchnorm_forward(Z2, gamma2, beta2, eps)
    A2 = softmax(Z2)

    ##################### Backward Prop #######################
    ################### Layer 2 ###############################
    dZ2 = dZ2 = A2 - Y  
    
    ############## Backward batch normalization ###############
    dZ2,dgamma2,dbeta2 = batchnorm_backward(dZ2, cache2)
    gamma2 = gamma2 - alpha2*dgamma2
    beta2 = beta2 - alpha2*dbeta2
    
    dW2 = (1/n)*np.dot(A1.T,dZ2)
    W2 = W2 - alpha2*dW2
    db2 = (1/n)*np.sum(dZ2)
    b2 = b2 - alpha2*db2
    
    ###################### Layer 1 ############################
    dA1 = np.dot(dZ2,W2)
    dZ1_temp = np.dot(dZ2,W2)
    dZ1 = relu_back(dZ1_temp,Z1)
    
    ############## Backward batch normalization ###############
    dZ1,dgamma1,dbeta1 = batchnorm_backward(dZ1, cache1)
    gamma1 = gamma1 - alpha1*dgamma1
    beta1 = beta1 - alpha1*dbeta1
    
    dW1 = (1/n)*np.dot(S,dZ1)
    W1 = W1 - alpha1*dW1
    db1 = (1/n)*np.sum(dZ1)
    b1 = b1 - alpha1*db1

    ##################### Cost calculation ####################
    cost.append(costfun(Y,A2))
    print(cost)
    
    ##################### Backward pooling ####################
    dS_prev = np.dot(W1,dZ1.T)
    dS = inv_flattening(dS_prev,a,l,m,o)
    dA_prev = pool_backward(dS, cache, mode = "max")

    #################### Backward RelU ########################
    dR1 = relu_back(dA_prev,R1)

    #################### Backwar convolution ##################
    dA, dW, db = conv_backward(Z, cache_conv)

    #################### Parameters Update ####################
    W = W - alpha*dW
    b = b - alpha*db

[8.517193191416238, 8.51610121572464, 8.515065486851206, 8.514086134192862, 8.513163287176768, 8.512297075241882, 8.511487627819363, 8.510735074311878, 8.510039544071734, 8.509401166377948, 8.50882007041208]
[8.517193191416238, 8.51610121572464, 8.515065486851206, 8.514086134192862, 8.513163287176768, 8.512297075241882, 8.511487627819363, 8.510735074311878, 8.510039544071734, 8.509401166377948, 8.50882007041208, 8.50829638523301]
[8.517193191416238, 8.51610121572464, 8.515065486851206, 8.514086134192862, 8.513163287176768, 8.512297075241882, 8.511487627819363, 8.510735074311878, 8.510039544071734, 8.509401166377948, 8.50882007041208, 8.50829638523301, 8.507830239750497]
[8.517193191416238, 8.51610121572464, 8.515065486851206, 8.514086134192862, 8.513163287176768, 8.512297075241882, 8.511487627819363, 8.510735074311878, 8.510039544071734, 8.509401166377948, 8.50882007041208, 8.50829638523301, 8.507830239750497, 8.507421762697671]
[8.517193191416238, 8.51610121572464, 8.515065486851206, 

In [40]:
######################## Accuracy Calculation ########################
A2[np.reshape(np.amax(A2,axis=1),(n,1))==A2] = 1
A2[np.reshape(np.amax(A2,axis=1),(n,1))!=A2] = 0

acc_mat = np.multiply(A2,Y)
acc = np.sum(acc_mat)/n

print("Train accuracy of the model is", acc)

Train accuracy of the model is 0.132


In [32]:
cost

[8.517193191416242,
 8.516101215724646,
 8.515065486851206,
 8.51408613419286,
 8.51316328717677,
 8.512297075241882,
 8.511487627819372,
 8.510735074311878,
 8.510039544071732,
 8.509401166377947,
 8.508820070412082,
 8.508296385233,
 8.5078302397505,
 8.507421762697668,
 8.507071082602288,
 8.50677832775693,
 8.506543626187995,
 8.506367105623616,
 8.506248893460402,
 8.506189116729066]